In [36]:
import requests               # Importa la librería para realizar solicitudes HTTP (GET, POST, etc.)
import json                   # Permite manejar archivos y datos en formato JSON
from IPython.display import JSON  # Sirve para mostrar objetos JSON de manera legible en entornos Jupyter
from keys import *            # Importa las claves o tokens almacenadas en el archivo "keys.py"


In [37]:
# be careful not to run too often as there is a maximum of 200 api calls per month
# (Ten cuidado de no ejecutar muy seguido ya que hay un máximo de 200 llamadas a la API por mes)

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/EDDB/2025-09-01T08:00/2025-09-01T20:00"
# URL del endpoint de la API donde se consultarán los vuelos de llegada al aeropuerto con código ICAO "EDDB"
# entre las 08:00 y las 20:00 del 1 de septiembre de 2025

querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}
# Parámetros de la solicitud que indican que se quiere obtener:
# - Información de los tramos del vuelo (withLeg)
# - Solo llegadas (direction)
# - Vuelos cancelados, compartidos, de carga o privados incluidos
# - Sin ubicación geográfica (withLocation=false)

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",  # Host del servicio de la API
    'x-rapidapi-key': "cf54169d5emshcaf0b84d309331fp103efejsnd8dfbdab47f6"                     # Clave de autenticación importada del archivo keys.py
    }

# Se realiza la solicitud GET a la API usando la librería requests, enviando URL, encabezados y parámetros
response = requests.request("GET", url, headers=headers, params=querystring)

In [38]:
# Se verifica si la respuesta tiene código 200 (solicitud exitosa)
if response.status_code == 200:
    new_data = response.json()  # Convierte la respuesta JSON de la API en un diccionario de Python

    try:
        # Intenta abrir el archivo "data.json" en modo lectura para agregar nuevos datos
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file)  # Carga los datos existentes del archivo
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        # Si el archivo no existe o está vacío/corrupto, crea una lista vacía
        existing_data = []

    # Agrega los nuevos datos obtenidos de la API a la lista de datos existente
    existing_data.append(new_data)

    # Abre el archivo en modo escritura para sobrescribir con los datos actualizados
    with open("data.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)  # Guarda el archivo con formato legible (indentación de 4)
        print("Data appended to data.json file.")      # Muestra mensaje de confirmación

# Si la solicitud no fue exitosa, muestra el código de error
else:
    print("Failed to retrieve data from the API. Status code:", response.status_code)

Data appended to data.json file.


In [39]:
flight_arrivals = response.json()  # Convierte nuevamente la respuesta en formato JSON y la guarda en la variable flight_arrivals

# Bucle que intenta recorrer la información de vuelos
for flight in flight_arrivals:
    # Accede al primer vuelo dentro de la lista de llegadas y muestra varios datos (no imprime, solo accede)
    flight_arrivals['arrivals'][0]['arrival']['scheduledTime']       # Hora programada de llegada
    flight_arrivals['arrivals'][0]['number']                         # Número de vuelo
    flight_arrivals['arrivals'][0]['arrival']['terminal']            # Terminal de llegada
    flight_arrivals['arrivals'][0]['departure']['airport']['name']   # Nombre del aeropuerto de salida
    flight_arrivals['arrivals'][0]['aircraft']['model']              # Modelo del avión
    flight_arrivals['arrivals'][0]['airline']['name']                # Nombre de la aerolínea

In [40]:
import pandas as pd          # Importa la librería pandas para manejar datos tabulares (DataFrames)
from numpy import nan        # Importa el valor NaN (Not a Number) para representar datos faltantes

flight_arrivals = response.json()  # Carga nuevamente los datos JSON de la respuesta en la variable flight_arrivals

# Define una función para extraer información de cada vuelo
def flight_extraction(flights):
    # Normaliza los datos JSON en formato tabular usando pandas.json_normalize
    flights_df = pd.json_normalize({
        'Scheduled_arrival_time': flights['arrival']['scheduledTime'],   # Hora programada de llegada
        'flight_number': flights['number'],                              # Número del vuelo
        'from' : flights['departure']['airport']['name'],                # Aeropuerto de origen
        'airline' : flights['airline']['name'],                          # Aerolínea
        'aircraft' : flights['aircraft']['model']                        # Modelo de avión
    })
    return(flights_df)  # Devuelve el DataFrame creado con los datos del vuelo

# Combina (concatena) todos los vuelos del JSON en un solo DataFrame usando comprensión de listas
flight_arrivals = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals['arrivals']]  # Aplica la función a cada vuelo en 'arrivals'
)

flight_arrivals.head()  # Muestra las primeras filas del DataFrame resultante

,flight_number,from,airline,aircraft,Scheduled_arrival_time.utc,Scheduled_arrival_time.local
0,DE 4095,Frankfurt-am-Main,Condor,Airbus A320,2025-09-01 05:55Z,2025-09-01 07:55+02:00
0,DE 2501,Dubai,Condor,Airbus A320 NEO,2025-09-01 06:25Z,2025-09-01 08:25+02:00
0,PC 5067,Antalya,Pegasus,Airbus A320 (Sharklets),2025-09-01 06:35Z,2025-09-01 08:35+02:00
0,JL 6899,Helsinki,JL,Airbus A319,2025-09-01 06:05Z,2025-09-01 08:05+02:00
0,AY 1431,Helsinki,Finnair,Airbus A319,2025-09-01 06:05Z,2025-09-01 08:05+02:00
